In [15]:
from korPdfRag import PdfRAGSystem

In [ ]:
# RAG 사용 객체 생성
rag_system = PdfRAGSystem()

# 인덱스 생성하기
rag_system.create_index()

Loading Korean embeddings model: intfloat/multilingual-e5-large


c:\Users\C117\miniconda3\envs\llmEnv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\C117\.cache\huggingface\hub\models--intfloat--multilingual-e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Creating index: pdf-manual-index
Index created successfully!


In [16]:
# PDF 문서 로드
pdf_path='cuckoo_레시피.pdf'

# 청킹된 결과를 만듭니다. 리턴타입이 Document 객체의 리스트 
chunks = rag_system.load_and_split_pdf(
    pdf_path=pdf_path,
    chunk_size=800,
    chunk_overlap=200
)

print(chunks[0])

Loading PDF: cuckoo_레시피.pdf
Loaded 56 pages, type: <class 'list'>
After filtering image-only pages: 56 pages
Split into 65 chunks
page_content='영
양
가
득
한
그
릇
밥
난이도별
한그릇 레시피' metadata={'source': 'cuckoo_레시피.pdf', 'file_path': 'cuckoo_레시피.pdf', 'page': 0, 'total_pages': 56, 'Author': 'imac1', 'CreationDate': "D:20231115110537+09'00'", 'Creator': 'QuarkXPress(R) 14.12', 'ModDate': "D:20231115110712+09'00'", 'Producer': 'QuarkXPress(R) 14.12', 'Title': '???? 1', 'XPressPrivate': '%%DocumentProcessColors: Cyan Magenta Yellow Black\n%%EndComments'}


In [10]:
print('chunks[0].id :',chunks[0].id)
print('chunks[0].metadata :',chunks[0].metadata)
print('chunks[0].page_content :',chunks[0].page_content)
print('chunks[0].type :',chunks[0].type)

chunks[0].id : None
chunks[0].metadata : {'source': 'cuckoo_레시피.pdf', 'file_path': 'cuckoo_레시피.pdf', 'page': 0, 'total_pages': 56, 'Author': 'imac1', 'CreationDate': "D:20231115110537+09'00'", 'Creator': 'QuarkXPress(R) 14.12', 'ModDate': "D:20231115110712+09'00'", 'Producer': 'QuarkXPress(R) 14.12', 'Title': '???? 1', 'XPressPrivate': '%%DocumentProcessColors: Cyan Magenta Yellow Black\n%%EndComments', 'text': '영\n양\n가\n득\n한\n그\n릇\n밥\n난이도별\n한그릇 레시피'}
chunks[0].page_content : 영
양
가
득
한
그
릇
밥
난이도별
한그릇 레시피
chunks[0].type : Document


In [11]:
print(len(chunks))

65


In [17]:
# 업서트
rag_system.create_vectorstore(chunks)

Creating vector store and embedding documents with Korean model...
Vector store created successfully!


In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
# 지정된 인덱스의 벡터값 가져오기
vectorstore = rag_system.load_vectorstore()

Loading existing vector store...
Vector store loaded!


In [20]:
# 임베딩된 검색(retriever) 객체 생성 : 파이프라인으로 사용
# docs = vector_store.similarity_search(query=question[0], k=5, namespace="wiki-ns1") 는 query 로 유사도 검색
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}          # 검색결과에서 유사도 순서로 5개 가져오기
)
        
# 한글 프롬프트 : docstring 에서는 f'' 없이 입력변수 {} 지정
template = """당신은 제품 매뉴얼 기반 질의응답 전문가입니다.
주어진  바탕으로 사용자의 질문에 정확하고 상세하게 답변해주세요.

답변 시 주의사항:
- 컨텍스트에 정보가 있으면 그것을 기반으로 답변하세요.
- 컨텍스트에 없는 내용은 "제공된 매뉴얼에서 해당 정보를 찾을 수 없습니다"라고 답변하세요.
- 한글로 명확하고 이해하기 쉽게 설명하세요.
- 필요하면 단계별 설명을 제공하세요.

컨텍스트:
{context}

질문: {question}

답변:"""

In [21]:
prompt = ChatPromptTemplate.from_template(template)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(model='gpt-4.1-mini',temperature=0)

# RAG 체인 구성
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain

{
  context: VectorStoreRetriever(tags=['PineconeVectorStore', 'KoreanEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000023189CB5700>, search_kwargs={'k': 5})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='당신은 제품 매뉴얼 기반 질의응답 전문가입니다.\n주어진  바탕으로 사용자의 질문에 정확하고 상세하게 답변해주세요.\n\n답변 시 주의사항:\n- 컨텍스트에 정보가 있으면 그것을 기반으로 답변하세요.\n- 컨텍스트에 없는 내용은 "제공된 매뉴얼에서 해당 정보를 찾을 수 없습니다"라고 답변하세요.\n- 한글로 명확하고 이해하기 쉽게 설명하세요.\n- 필요하면 단계별 설명을 제공하세요.\n\n컨텍스트:\n{context}\n\n질문: {question}\n\n답변:'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000230461233B0>, async_client=<openai.resources.chat.completions.completions.AsyncComplet

In [22]:
# questions = [
# "스팸김치날치알 요리방법",
# "스팸김치날치알 재료",
# ]

questions = [
"tomato bacon 요리방법",
"tomato bacon 재료",
]
for question in questions:
    print(f"\n질문: {question}")
    print("-" * 60)
    
    # 답변만 받기
    answer = rag_chain.invoke(question)
    print(f"답변: {answer}\n")


질문: tomato bacon 요리방법
------------------------------------------------------------
답변: tomato bacon 요리방법은 다음과 같습니다.

1. 베이컨 6줄을 1cm 두께로 썰고, 방울토마토 10개는 깨끗이 씻어 준비합니다.
2. 깨끗이 씻은 쌀 2컵(300g)을 내솥에 넣고, 무압백미 물눈금 2까지 물을 부어줍니다.
3. 그 위에 썰어둔 베이컨을 올립니다.
4. 뚜껑을 닫고 핸들 손잡이를 무압모드로 돌립니다.
5. 좌/우 버튼으로 [영양밥-1단계] 메뉴를 선택한 후 시작 버튼을 눌러 취사를 시작합니다.
6. 취사 시작 후 20분이 지나면 오픈쿠킹 버튼을 눌러 뚜껑을 열고, 방울토마토를 넣은 뒤 뚜껑을 닫습니다. (뚜껑을 닫으면 취사가 계속 진행됩니다.)
7. 취사가 완료되면 올리브유 1큰술을 두르고, 파슬리 약간을 올려 마무리합니다.

주의사항: 방울토마토는 반드시 오픈쿠킹 기능을 활용하여 취사 중간에 넣어야 하며, 지정된 용량을 초과하지 않도록 주의하세요. 끓어 넘침이 발생할 수 있습니다.


질문: tomato bacon 재료
------------------------------------------------------------
답변: tomato bacon 요리에 필요한 재료는 다음과 같습니다.

- 쌀 2컵 (300g)  
- 베이컨 6줄  
- 방울토마토 10개  
- 올리브유 1큰술  
- 파슬리 약간  

이 재료들을 사용하여 조리하시면 됩니다.



In [ ]:
# KoreanEmbeddings 의 속성 : embeddingsa
# PdfRAGSystem 의 속성 
# index_name 문자열, pc(파인콘),embeddings,llm,vectorstore 객체
'''
PDF 메뉴얼 기반 RAG 시스템 - Pinecone + LangChain 1.x

한글 특화 임베딩 모델 + 이미지 무시
- 백터DB 에 저장할 문서(텍스트)는 PDF 에서 가져옵니다.
- 파인콘에 백터DB 저장
    - 임베딩 모델, 인덱스 이름 결정
    - PineconeVectorStore : vectorstore 객체로 저장

- 파인콘에 저장된 index 에서 검색(유사도 검색)
    - PineconeVectorStore : vectorstore 객체에서 검색
    - 조회 결과로 만들어진 텍스트는 llm 에게 전달하여 최종 응답 메세지 완성
'''

In [24]:
#  여기까지는 테스트.
# 실제 실행은 main 함수로
import korPdfRag

questions = [
"구우치즈밥 요리방법",
"구우치즈밥 재료",
]
korPdfRag.main(False, None,'pdf-manual-index',questions)
# False 는 다시 upsert 하지 않도록

Loading Korean embeddings model: intfloat/multilingual-e5-large
Index 'pdf-manual-index' already exists.
Loading existing vector store...
Vector store loaded!

PDF 매뉴얼 RAG 시스템 (한글 특화)

질문: 구우치즈밥 요리방법
------------------------------------------------------------
답변: 구우치즈밥 요리방법은 다음과 같습니다.

1. 깨끗이 씻은 쌀 2컵(300g)을 내솥에 넣고 무압백미 물눈금 2까지 물을 부어줍니다.  
2. 뚜껑을 닫고 핸들 손잡이를 무압모드로 돌려줍니다.  
3. 좌/우 버튼으로 [영양밥-1단계] 메뉴를 선택한 후 시작 버튼을 눌러 취사를 시작합니다.  
4. 취사 시작 20분 뒤 오픈쿠킹 버튼을 눌러 뚜껑을 열고 레토르트 카레 400g을 부어 잘 섞은 후, 밥 한가운데를 오목하게 만들어 계란 1개를 깨서 넣습니다.  
5. 그 위에 슬라이스 치즈 1장과 모짜렐라 치즈 50g을 올리고 뚜껑을 닫으면 취사가 계속 진행됩니다.  
6. 취사가 완료되면 뚜껑을 열어 파슬리를 약간 뿌려 마무리합니다.

**주의사항:**  
- 카레, 치즈, 계란은 오픈쿠킹 기능을 활용하여 취사 중간에 넣어야 하며, 반드시 레시피에 명시된 투입 시점을 지켜주세요.  
- 지정된 용량을 초과하지 마십시오. 끓어 넘침이 발생할 수 있습니다.  

이상으로 구우치즈밥의 상세한 요리방법입니다.


질문: 구우치즈밥 재료
------------------------------------------------------------
답변: 구우치즈밥 재료는 다음과 같습니다.

- 쌀 2컵 (300g)  
- 레토르트 카레 400g  
- 슬라이스 치즈 1장  
- 모짜렐라 치즈 50g  
- 계란 1개  
- 파슬리 약간  

이 재료들을 사용하여 구우치즈밥을 만드실 수 있습니다.

